In [ ]:
from gdeltdoc import GdeltDoc, Filters
from datetime import datetime, timedelta
import pandas as pd
import openai

ModuleNotFoundError: ignored

In [ ]:
#OpenAI API Calling
def getAnswer(conversation, temperature=1, top_p=1, maxT=800):
    response = openai.ChatCompletion.create(
        engine="gpt-35-turbo",  # Elige el motor adecuado
        messages=conversation,
        temperature=temperature,  # Controla la creatividad de las respuestas
        top_p = top_p,
        max_tokens = maxT
    )
    return response

In [ ]:
#Get dates for making searches
def getSearchDates():
    dateNow = datetime.now()
    customFormat = "%Y-%m-%d"

    endDate = dateNow.strftime(customFormat)
    startDate = (datetime.strptime(endDate, customFormat) - timedelta(days=91)).strftime(customFormat)

    # Lista para almacenar pares de fechas
    date_pairs = []

    endDate = datetime.strptime(endDate, customFormat)
    startDate = datetime.strptime(startDate, customFormat)

    while startDate <= endDate:
        end_of_interval = startDate + timedelta(days=5)

        if end_of_interval > endDate:
            end_of_interval = endDate

        date_pairs.append((startDate.strftime(customFormat), end_of_interval.strftime(customFormat)))
        startDate += timedelta(days=6)  # Avanzar 5 días para el próximo intervalo

    return date_pairs

In [ ]:
#Retrieving Documents
gd = GdeltDoc()

#Getting date range
calculatedDates = getSearchDates()

#Save retrieved results
searched_results = pd.DataFrame()
timelines = pd.DataFrame()

#for each date calculated
for date in calculatedDates:

    # Creating looking for objetc
    f = Filters(
        keyword="Tuberculosis",
        start_date=date[0],
        end_date=date[1]
    )

    print("Making API callings with parameters:\n---  ", f, "  ---")

    # API calling
    articles = gd.article_search(f)

    # Get a timeline of the number of articles matching the filters
    timeline = gd.timeline_search("timelinevol", f)

    # Concatening data into current dataframes
    searched_results = pd.concat([searched_results, articles], ignore_index=True)
    timelines = pd.concat([timelines, timeline], ignore_index=True)

Making API callings with parameters:
---   <gdeltdoc.filters.Filters object at 0x000000000A35FD60>   ---
Making API callings with parameters:
---   <gdeltdoc.filters.Filters object at 0x000000000A35FBE0>   ---
Making API callings with parameters:
---   <gdeltdoc.filters.Filters object at 0x000000000A35F490>   ---
Making API callings with parameters:
---   <gdeltdoc.filters.Filters object at 0x000000000769AFA0>   ---
Making API callings with parameters:
---   <gdeltdoc.filters.Filters object at 0x00000000078D75E0>   ---
Making API callings with parameters:
---   <gdeltdoc.filters.Filters object at 0x000000000769AF10>   ---
Making API callings with parameters:
---   <gdeltdoc.filters.Filters object at 0x000000000A35FE20>   ---
Making API callings with parameters:
---   <gdeltdoc.filters.Filters object at 0x000000000A35FCD0>   ---
Making API callings with parameters:
---   <gdeltdoc.filters.Filters object at 0x000000000A35F6A0>   ---
Making API callings with parameters:
---   <gdeltdoc.fi

In [ ]:
#Comparing shapes of each dataframe
print("docs", searched_results.shape, "timelines", timelines.shape)

docs (3812, 8) timelines (1884, 2)


In [ ]:
enfermedades = []
titleNumber = 1
# Making prompting for knowing titles themes
initialPrompt = """
Tienes que leer el [título] de noticias acerca de enfermedades que te digo al final del prompt y generar tres palabras clave que expliquen cada [título].
Para ello tienes que hacer lo siguiente:
1 - Lee cada [título] y si está en otro idioma, tradúcelo al español. No es necesario que lo digas después.
2 - Interpreta la temática de cada [título] y clasificalo para ver acerca de que habla. No es neesario que digas el título
3 - Devuelve esa interpretación que has hecho, generando tres palabras para saber de que va cada uno. Para ello puedes hacer lo siguiente:
    3.1 - Fíjate la enfermedad de la que habla
    3.2 - Fíjate para ver a quién afecta la enfermedad
    3.3 - Fíjate para ver las causas de la enfermedad
4 - El [titulo] que tienes que interpretar es el siguiente. [Titulo]:
"""
print(" **** Total títulos a analizar:", searched_results.title.shape, " **** \n")
# Making Open AI API calls
for title in searched_results.title:
    conversation = [{"role": "system", "content": initialPrompt+str(title)}]
    response = getAnswer(conversation)
    enfermedades.append(response)
    if(titleNumber % 1 == 0):
        print("- Titulo número:", titleNumber, "\n")
    #Limitation to 20 API calls for working faster
    if(titleNumber == 21):
        break
    titleNumber +=1

 **** Total títulos a analizar: (3812,)  **** 

- Titulo número: 1 

- Titulo número: 2 

- Titulo número: 3 

- Titulo número: 4 

- Titulo número: 5 

- Titulo número: 6 

- Titulo número: 7 

- Titulo número: 8 

- Titulo número: 9 

- Titulo número: 10 

- Titulo número: 11 

- Titulo número: 12 

- Titulo número: 13 

- Titulo número: 14 

- Titulo número: 15 

- Titulo número: 16 

- Titulo número: 17 

- Titulo número: 18 

- Titulo número: 19 

- Titulo número: 20 

- Titulo número: 21 



In [ ]:
for i in range(0, len(enfermedades)):
     try:
            print("\n*****************************************************************************************************************************\n","Titulo: ", i+1, "\n\n", enfermedades[i]["choices"][0]["message"]["content"])
     except Exception as e:
        print("*****************************************************************************************************************************\n")
        print(f"\033[1;31m Error al volcar información en el elemento número: '{i+1}': {str(e)} \033[0m\n")


*****************************************************************************************************************************
 Titulo:  1 

 1. Vacuna contra la tuberculosis: poderosa arma contra la tuberculosis en el horizonte.

2. Temática: enfermedad de la tuberculosis y la posible llegada de una vacuna para combatirla.

3. Palabras clave:
- Tuberculosis
- Vacuna
- Prevención

*****************************************************************************************************************************
 Titulo:  2 

 Temática: Enfermedad del tuberculosis
1. Ossos: Dolor, fracturas, debilidad.
2. Rins: Insuficiencia renal, pérdida de función renal, dolor.
3. Cérebro: Confusión, pérdida de memoria, trastornos cognitivos.

*****************************************************************************************************************************
 Titulo:  3 

 Interpretación: Se trata de una mejora en el tratamiento de la tuberculosis con el fin de hacerlo más llevadero para el pacient